<a href="https://colab.research.google.com/github/vpysmennyi/machinelearning-learning/blob/main/hometask5_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR10

In [7]:
import numpy as np
import torch
import torchvision.transforms as transforms 

from torch import nn
from torch.utils.data import DataLoader
from torch.optim import SGD
from torchvision.datasets import CIFAR10

train_ds = CIFAR10('.', train=True, transform=transforms.ToTensor(), download=True)
val_ds = CIFAR10('.', train=False, transform=transforms.ToTensor(), download=True)

batch_size = 128
hidden_size = 128

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

class BatchNorm(nn.Module):
  def __init__(self, n_features, momentum=0.1, eps=1e-5):
    super(BatchNorm, self).__init__()
    self.momentum = momentum
    self.eps = eps
    
    # afine parameters
    self.weight = nn.Parameter(torch.Tensor(n_features))
    self.bias = nn.Parameter(torch.Tensor(n_features))

    # running statistics
    self.register_buffer('running_mean', torch.zeros(n_features))
    self.register_buffer('running_var', torch.zeros(n_features))

    self.reset_parameters()

  def reset_parameters(self):
    nn.init.ones_(self.weight)
    nn.init.zeros_(self.bias)
    self.running_mean.zero_()
    self.running_var.fill_(1)

  def forward(self, x):
    if self.training:
      batch_mean = torch.mean(x, dim=0)
      batch_var = torch.var(x, dim=0)
      
      x = (x - batch_mean) / torch.sqrt(batch_var + self.eps)

      self.running_mean = self.momentum * batch_mean.detach() + (1 - self.momentum) * self.running_mean
      self.running_var = self.momentum * batch_var.detach() + (1 - self.momentum) * self.running_var
    else:
      x = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

    x = self.weight * x + self.bias
    return x

class FFN(nn.Module):
  def __init__(self):
    super(FFN, self).__init__()

    self.net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(32*32*3, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, 10)
    )

  def forward(self, x):
    return self.net(x)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
torch.cuda.init()
network = FFN()
opt = SGD(network.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 50

for epoch_ind in range(n_epochs):
  train_loss, train_acc = [], []
  
  network.train()
  for (X, Y) in train_dl:
    opt.zero_grad()
    Yp = network(X)
    loss = loss_fn(Yp, Y)
    loss.backward()
    opt.step()

    accuracy = (Yp.argmax(1) == Y).float().mean()
    train_acc.append(accuracy.detach())
    train_loss.append(loss.detach())

  print(f"{epoch_ind} Train loss: {torch.stack(train_loss).mean():.6f}\t acc: {torch.stack(train_acc).mean():.4f}")

  network.eval()
  val_loss, val_acc = [], []
  for (X, Y) in val_dl:
    Yp = network(X)
    
    accuracy = (Yp.argmax(1) == Y).float().mean()
    val_acc.append(accuracy)

    loss = loss_fn(Yp, Y)
    val_loss.append(loss)

  print(f"{epoch_ind} Val loss: {torch.stack(val_loss).mean():.6f}\t acc: {torch.stack(val_acc).mean():.4f}")

0 Train loss: 1.990762	 acc: 0.2915
0 Val loss: 1.846510	 acc: 0.3478
1 Train loss: 1.779000	 acc: 0.3769
1 Val loss: 1.731272	 acc: 0.3925
2 Train loss: 1.695809	 acc: 0.4091
2 Val loss: 1.656721	 acc: 0.4233
3 Train loss: 1.643051	 acc: 0.4269
3 Val loss: 1.622609	 acc: 0.4362
4 Train loss: 1.602857	 acc: 0.4424
4 Val loss: 1.578662	 acc: 0.4498
5 Train loss: 1.572059	 acc: 0.4518
5 Val loss: 1.547497	 acc: 0.4570
6 Train loss: 1.539851	 acc: 0.4598
6 Val loss: 1.514074	 acc: 0.4683
7 Train loss: 1.514478	 acc: 0.4688
7 Val loss: 1.498815	 acc: 0.4763
8 Train loss: 1.490176	 acc: 0.4788
8 Val loss: 1.501155	 acc: 0.4734
9 Train loss: 1.471220	 acc: 0.4860
9 Val loss: 1.472072	 acc: 0.4830
10 Train loss: 1.450944	 acc: 0.4918
10 Val loss: 1.430384	 acc: 0.4981
11 Train loss: 1.432522	 acc: 0.4978
11 Val loss: 1.419469	 acc: 0.5031
12 Train loss: 1.419272	 acc: 0.5031
12 Val loss: 1.412776	 acc: 0.5052
13 Train loss: 1.402765	 acc: 0.5102
13 Val loss: 1.389674	 acc: 0.5142
14 Train los

batch_size = 128,
hidden_size = 128,
ReLU,
LR =0.001,
epochs = 50,
Batch Normalization

0 Train loss: 1.990762	 acc: 0.2915
0 Val loss: 1.846510	 acc: 0.3478
1 Train loss: 1.779000	 acc: 0.3769
1 Val loss: 1.731272	 acc: 0.3925
2 Train loss: 1.695809	 acc: 0.4091
2 Val loss: 1.656721	 acc: 0.4233
3 Train loss: 1.643051	 acc: 0.4269
3 Val loss: 1.622609	 acc: 0.4362
4 Train loss: 1.602857	 acc: 0.4424
4 Val loss: 1.578662	 acc: 0.4498
5 Train loss: 1.572059	 acc: 0.4518
5 Val loss: 1.547497	 acc: 0.4570
6 Train loss: 1.539851	 acc: 0.4598
6 Val loss: 1.514074	 acc: 0.4683
7 Train loss: 1.514478	 acc: 0.4688
7 Val loss: 1.498815	 acc: 0.4763
8 Train loss: 1.490176	 acc: 0.4788
8 Val loss: 1.501155	 acc: 0.4734
9 Train loss: 1.471220	 acc: 0.4860
9 Val loss: 1.472072	 acc: 0.4830
10 Train loss: 1.450944	 acc: 0.4918
10 Val loss: 1.430384	 acc: 0.4981
11 Train loss: 1.432522	 acc: 0.4978
11 Val loss: 1.419469	 acc: 0.5031
12 Train loss: 1.419272	 acc: 0.5031
12 Val loss: 1.412776	 acc: 0.5052
13 Train loss: 1.402765	 acc: 0.5102
13 Val loss: 1.389674	 acc: 0.5142
14 Train loss: 1.389489	 acc: 0.5137
14 Val loss: 1.385977	 acc: 0.5130
15 Train loss: 1.372732	 acc: 0.5187
15 Val loss: 1.421266	 acc: 0.5007
16 Train loss: 1.359489	 acc: 0.5248
16 Val loss: 1.363912	 acc: 0.5219
17 Train loss: 1.345128	 acc: 0.5292
17 Val loss: 1.341355	 acc: 0.5283
18 Train loss: 1.332304	 acc: 0.5350
18 Val loss: 1.362472	 acc: 0.5179
19 Train loss: 1.319085	 acc: 0.5387
19 Val loss: 1.313122	 acc: 0.5404
20 Train loss: 1.309508	 acc: 0.5419
20 Val loss: 1.300110	 acc: 0.5456
21 Train loss: 1.298185	 acc: 0.5442
21 Val loss: 1.306535	 acc: 0.5400
22 Train loss: 1.285946	 acc: 0.5499
22 Val loss: 1.274118	 acc: 0.5545
23 Train loss: 1.274307	 acc: 0.5557
23 Val loss: 1.274320	 acc: 0.5520
24 Train loss: 1.264695	 acc: 0.5589
24 Val loss: 1.307747	 acc: 0.5415
25 Train loss: 1.252475	 acc: 0.5637
25 Val loss: 1.304033	 acc: 0.5400
26 Train loss: 1.243024	 acc: 0.5646
26 Val loss: 1.268712	 acc: 0.5500
27 Train loss: 1.232908	 acc: 0.5708
27 Val loss: 1.254438	 acc: 0.5603
28 Train loss: 1.221766	 acc: 0.5734
28 Val loss: 1.234352	 acc: 0.5709
29 Train loss: 1.215171	 acc: 0.5744
29 Val loss: 1.214465	 acc: 0.5741
30 Train loss: 1.204532	 acc: 0.5802
30 Val loss: 1.283087	 acc: 0.5537
31 Train loss: 1.195120	 acc: 0.5821
31 Val loss: 1.279645	 acc: 0.5550
32 Train loss: 1.187408	 acc: 0.5840
32 Val loss: 1.216128	 acc: 0.5724
33 Train loss: 1.178394	 acc: 0.5882
33 Val loss: 1.176677	 acc: 0.5861
34 Train loss: 1.168763	 acc: 0.5916
34 Val loss: 1.197605	 acc: 0.5799
35 Train loss: 1.158991	 acc: 0.5955
35 Val loss: 1.319411	 acc: 0.5289
36 Train loss: 1.152077	 acc: 0.5980
36 Val loss: 1.213666	 acc: 0.5773
37 Train loss: 1.144534	 acc: 0.5991
37 Val loss: 1.219435	 acc: 0.5688
38 Train loss: 1.135619	 acc: 0.6023
38 Val loss: 1.135689	 acc: 0.6032
39 Train loss: 1.129149	 acc: 0.6059
39 Val loss: 1.174253	 acc: 0.5869
40 Train loss: 1.120166	 acc: 0.6097
40 Val loss: 1.110891	 acc: 0.6108
41 Train loss: 1.110771	 acc: 0.6110
41 Val loss: 1.258806	 acc: 0.5482
42 Train loss: 1.101712	 acc: 0.6169
42 Val loss: 1.197847	 acc: 0.5762
43 Train loss: 1.095484	 acc: 0.6208
43 Val loss: 1.256672	 acc: 0.5547
44 Train loss: 1.088135	 acc: 0.6182
44 Val loss: 1.099282	 acc: 0.6174
45 Train loss: 1.078006	 acc: 0.6244
45 Val loss: 1.133403	 acc: 0.6018
46 Train loss: 1.071571	 acc: 0.6248
46 Val loss: 1.134904	 acc: 0.6011
47 Train loss: 1.065476	 acc: 0.6262
47 Val loss: 1.161636	 acc: 0.5844
48 Train loss: 1.058355	 acc: 0.6297
48 Val loss: 1.100878	 acc: 0.6143
49 Train loss: 1.050132	 acc: 0.6327
49 Val loss: 1.147772	 acc: 0.5989